In [57]:
import lazypredict
import pandas as pd
from sklearn.preprocessing import RobustScaler
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from xgboost import XGBClassifier

In [58]:
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
df_X_last = pd.read_csv("data/X_train_features.csv", index_col=0)
df_X_richard = pd.read_csv("data/template_features.csv", index_col=0)
#df_X_tim = pd.read_csv("full_waveform_features.csv", index_col=0)
df_X_pyHRV = pd.read_csv("data/pyHRV_features.csv", index_col=0)
df_X_hrv_analysis = pd.read_csv("data/hrv-analysis_features.csv", index_col=0)
df_X = pd.concat((df_X_richard,df_X_pyHRV,df_X_hrv_analysis),axis=1)
# df_X = pd.concat((df_X, df_X_last),axis=1)
df_Y = pd.read_csv("data/Y_train.csv", index_col="id")

In [59]:
df_X_template_test = pd.read_csv("data/template_features_test.csv", index_col=0)
#df_X_tim = pd.read_csv("full_waveform_features.csv", index_col=0)
df_X_pyHRV_test = pd.read_csv("data/pyHRV_features_test.csv", index_col=0)
df_X_hrv_analysis_test = pd.read_csv("data/hrv-analysis_features_test.csv", index_col=0)
df_X_test = pd.concat((df_X_template_test,df_X_pyHRV_test,df_X_hrv_analysis_test), axis=1)

Remove highly correlated features

In [60]:
def rm_corr(X, X_test):
    corr_matrix = X.corr().abs()

    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    # Find features with correlation greater than 0.9
    to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]
    print("Removed columns: ", len(to_drop))
    # Drop features 
    X.drop(to_drop, axis=1, inplace=True)
    X_test.drop(to_drop, axis=1, inplace=True)
    
    return X, X_test

In [61]:
df_X, df_X_test = rm_corr(df_X, df_X_test)

Removed columns:  22


Normalize

In [62]:
df_X.replace([np.inf, -np.inf], np.nan, inplace=True)
df_X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
transformer = RobustScaler()
X = transformer.fit_transform(df_X)
X_test = transformer.transform(df_X_test)

In [63]:
X.shape

(5117, 53)

Impute missing values

In [64]:
imp_median = SimpleImputer(missing_values=np.nan, strategy='median')
X = imp_median.fit_transform(X)
X_test = imp_median.transform(X_test)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, df_Y,stratify=df_Y,test_size=.2,random_state=0)
clf = LazyClassifier(verbose=0, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:33<00:00,  1.14s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
XGBClassifier,0.83,0.70,None,0.82,4.29
LGBMClassifier,0.83,0.69,None,0.82,1.53
LinearDiscriminantAnalysis,0.77,0.67,None,0.76,0.09
BaggingClassifier,0.80,0.67,None,0.80,2.17
AdaBoostClassifier,0.76,0.65,None,0.75,1.27
LogisticRegression,0.79,0.65,None,0.78,0.18
RandomForestClassifier,0.81,0.65,None,0.81,2.81
ExtraTreesClassifier,0.82,0.63,None,0.81,1.00
NearestCentroid,0.66,0.63,None,0.66,0.03


In [66]:
clf = XGBClassifier()
clf.fit(X, df_Y)
y_

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', ...)